In [1]:
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv("data/data_01.csv",sep="|")

In [4]:
df.head()

,Date,Title,Numac,Link FR,Link NL,nl_body,fr_body
0,14/01/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,NLFR belgiëlex.be - Kruispuntbank Wetgeving...,FRNL belgiquelex.be - Banque Carrefour de l...
1,16/01/2020,SERVICE PUBLIC FEDERAL INTERIEUR\n10 DECEMBRE ...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,NLFRDE ELI - Navigatie systeem via een Europes...,FRNLDE ELI - Système de navigation par identif...
2,16/01/2020,SERVICE PUBLIC FEDERAL FINANCES\n7 DECEMBRE 20...,2.020040e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,NLFRDE ELI - Navigatie systeem via een Europes...,FRNLDE ELI - Système de navigation par identif...
3,24/01/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,NLFR eindeeerste woordlaatste woordPublicat...,FRNL finpremier motdernier motPublié le : 2...
4,28/01/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministratio...,2.020010e+09,http://www.ejustice.just.fgov.be/cgi/article_b...,http://www.ejustice.just.fgov.be/cgi/article_b...,NLFR eindeeerste woordlaatste woordPublicat...,FRNL finpremier motdernier motPublié le : 2...


In [5]:
df['fr_body'][0]

"FRNL belgiquelex.be \xa0\xa0-\xa0\xa0Banque Carrefour de la législation  Conseil d'Etat ELI - Système de navigation par identifiant européen de la législation http://www.ejustice.just.fgov.be/eli/arrete/2020/01/08/2020010053/moniteur   finpremier motdernier motPublié le : 2020-01-14Numac : 2020010053REGION DE BRUXELLES-CAPITALE 8 JANVIER 2020. - Arrêté ministériel établissant les modèles de formulaire visé à l'article 8, § 1er, de l'arrêté du Gouvernement de la Région de Bruxelles-Capitale du 19 janvier 2017 établissant les modalités du régime préférentiel applicable aux droits de succession en cas de transmission des entreprises familiales et des sociétés familiales Le Ministre du Gouvernement de la Région \nde Bruxelles-Capitale, chargé des Finances et du Budget,Vu le Code des droits de succession, \nl'article 60bis/2, § 1er, inséré par l'ordonnance du 12 décembre 2016 portant \nla deuxième partie de la réforme fiscale;Vu l'arrêté du Gouvernement de la Région de Bruxelles \nCapitale

In [12]:
text = "".join(df['fr_body'][0])

In [13]:
from translate import Translator 

translator = Translator(from_lang="french", to_lang='english')
translation = translator.translate(text)
translation

'QUERY LENGTH LIMIT EXCEDEED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEDEED. MAX ALLOWED QUERY : 500 CHARS QUERY LENGTH LIMIT EXCEDEED. MAX ALLOWED QUERY : 500 CHARS Michel DAERDEN Vice-Minister-President and Minister for the Budget and Finance (Frenchspeaking Community)'

In [16]:
from deep_translator import GoogleTranslator

to_translate = text
translated = GoogleTranslator(source='french', target='english').translate(to_translate)
translated

"FRNL belgiquelex.be - Crossroads bank for legislation Council of State ELI - Navigation system by European legislation identifier http://www.ejustice.just.fgov.be/eli/arrete/2020/01/08/2020010053 /monitor endfirst wordlast wordPublished on: 2020-01-14Numac: 2020010053BRUSSELS-CAPITAL REGION 8 JANUARY 2020. - Ministerial decree establishing the model forms referred to in Article 8, § 1, of the decree of the Government of the Region of Brussels-Capital of 19 January 2017 establishing the terms of the preferential regime applicable to inheritance tax in the event of the transfer of family businesses and family companies The Minister of the Government of the Region\nof Brussels-Capital, in charge of Finance and Budget, Having regard to the Code of inheritance rights,\nArticle 60bis/2, § 1, inserted by the order of December 12, 2016 on\nthe second part of the tax reform; Having regard to the decree of the Government of the Region of Brussels\nCapital of January 19, 2017 establishing the te